SVD Image Compression Demo for HW in EECS 551  
??? Original version by ???  
2021-10-28 Julia 1.6.3 updates by Jeff Fessler (eliminate WebIO and Interact)

In [ ]:
# packages used; you may need to add some of these
using LinearAlgebra: svd, rank, Diagonal
using FileIO
using ColorTypes
using Images # if you have issues with this package, then comment it out
#using Interact
using Plots; default(markerstrokecolor=:auto)
println("Ready!")

In [ ]:
# Load and display an image
image_color = load("mit.jpg") # Replace with your image -- put in the same directory as this notebook

In [ ]:
# Convert to gray scale for simplicity
@show size(image_color)
image_gray = Gray.(image_color)

In [ ]:
# Convert to matrix of values to perform SVD
image_data = Float32.(image_gray)
typeof(image_gray)

In [ ]:
# Compute SVD and rank of image matrix
U, σ, V = svd(image_data)
r = rank(Diagonal(σ)) # efficient way to find rank given singular values!
println("Rank of image matrix = $r out of $(length(σ))")

Let $A$ be an $M \times N$ matrix having SVD

$$A = \sum_{i=1}^r \sigma_i u_i v_i',$$

where $r$ is the rank of $A$. Consider the optimization problem:

$$ \hat{A}_K = \arg \min_{X} \| A - X \|_F, \textrm{ subject to } \mathrm{rank}(X) ≤ K.$$

Via the famous Eckart-Young theorem, we have that

$$ \hat{A}_K = \sum_{i=1}^{\min(K,r)} \sigma_i u_i v_i'.$$

Mirsky showed that if we replace the Frobenius norm
in the above optimization problem by *any* unitarily invariant norm
(e.g., the operator norm, or the nuclear norm),
then the solution remains the same!

This notebook explores low-rank approximations
of an $M \times N$ sized (grayscale) image
viewed as an $M \times N$ matrix.

In [ ]:
# function to compute approximation error for a k-term approximation
# to a matrix with singular values in the vector σ
function svd_rank_k_approx_error(σ, k)
    return sum(σ[(k+1):end].^2) / sum(σ.^2)
end

In [ ]:
# Compute relative errors and plot them as a function of k
relative_error_fun = k -> svd_rank_k_approx_error(σ, k)
relative_error = relative_error_fun.(0:r)

# Plot errors
scatter(0:r, 100*relative_error, label = "",
    ylabel = "Relative error (percentage)",
    xlabel = "Rank of approximation",
	ylim = [0,12], # error at 0 is 100%
)

In [ ]:
# Examine approximation quality vs k.
scatter(0:r, 100 * (1 .- relative_error),
    label = "(1 - relative error) x 100%",
    ylabel = "Approximation quality (%)",
    xlabel = "Rank of approximation",
    xlim = (0, r + 0.5), # adjust if needed
    ylim = (0, 100),
)

In [ ]:
# See how values of k affect the visual quality of the approximation.
klist = [1, 10, 100, 800] # experiment here
ps = Array{Any}(undef, length(klist))
#@manipulate for k = klist # issues with WebIO
for (i, k) in enumerate(klist)
    image_k_matrix = U[:,1:k] * Diagonal(σ[1:k]) * V[:,1:k]'
    ps[i] = heatmap(image_k_matrix, color = :grays,
        yflip = :true, ticks = [],
        title = "K=$k, error = $(round(100*relative_error[k+1],digits=2)) %."
    )
end
plot(ps...)

In [ ]:
# plot singular values of the image
scatter(σ, label = :none,
    title = "Singular values of the image matrix",
    xlabel = "Index",
    ylabel = "Singular value",
)

In [ ]:
# Zoom in to see largest 50 singular values
ntop = 50
scatter(σ, label = :none,
    title = "Singular values of the image matrix",
    xlabel = "Index",
    ylabel = "Singular value",
    xlim = (0, ntop + 0.5),
)

Probably your image matrix seems to have the spectral signature
of a low-rank-signal-plus-noise type of matrix.
In this problem we do "better" in terms of approximating the image
as the rank parameter increases.
This makes it an approximation type problem.

If we were to add noise to this image
and then measure error relative to the noise-free image,
then we would see that
if the noise is large enough
then the error is best
when the approximation rank is less than the rank of signal-plus-noise matrix;
that would be a denoising problem
and insights from random matrix theory are invaluable there.
Determining whether one is operating in a denoising setup
versus an approximation setup is not always straightforward.
We say more to say about this later.

In [ ]:
# ## Optional below here
#
# Extract individual RGB channels
using Images: channelview, rawview
image_mtx = rawview(channelview(image_color)).data
red_image = Float32.(image_mtx[1,:,:])
green_image = Float32.(image_mtx[2,:,:])
blue_image = Float32.(image_mtx[3,:,:])

#### Optional question

What if we perform low-rank approximations of R, G and B matrices separately
and then combine them?
Is that approach "better" in any sense?

#### Optional question

What if we did the SVD on "patches" of the image
instead of the entire image?
A [p,q] patch of an image is a p x q rectangular subimage taken from the image.
In the patch-based approach one can imagine slicing the image up
into lots of (non-overlapping, for starters) patches,
computing an SVD of the individual patches,
and stitching them back together.
How does that work?
Try it and let us know!
You'll have to implement (or find) a Julia analog of
https://www.mathworks.com/help/images/ref/im2col.html